In [1]:
import chess.pgn
from IPython.display import SVG, display#

#pgnファイルの読み込み
pgn = open("13.pgn")
game = chess.pgn.read_game(pgn)

# チェスボードを初期化
board = game.board()

# 最初の手から順番に各手のチェスボードを図示して表示
for move_number, move in enumerate(game.mainline_moves(), start=1):
    # 手を実行
    board.push(move)
    
    # チェスボードを図示して表示
    #display(SVG(chess.svg.board(board=board)))
    
    # オプション: 各手の情報を表示
    print(f"Move {move_number}: {move.uci()}")
    print("FEN:", board.fen())


In [8]:
#ネットワーク化処理
import numpy as np
from pprint import pprint
board_list = [[] for _ in range(8)]
for row_i, row in enumerate(board.fen().split(" ")[0].split("/")):
    # print(row)
    for char in row:
        # print(char)
        try:
            integer = int(char)
            for none_i in range(integer):
                board_list[row_i].append("")
            # print(integer)
        except:
            board_list[row_i].append(char)
pprint(board_list)
batch_size = 1024
piece_list = ['p', 'k', 'b', 'r', 'q', 'k']
COLORS = [WHITE, BLACK] = range(2)
for color in COLORS:
    for piece in piece_list:
        if color == WHITE:
            char = piece.upper()
        else:
            char = piece
        piece_array = [[1 if char == board_list[y][x] else 0 for x in range(8)] for y in range(8)]
        pprint(piece_array)
        print()
        piece_array=np.array(piece_array)
        x = torch.from_numpy(x.astype(np.int8)).clone()

[['', '', '', '', '', 'r', 'k', ''],
 ['p', '', 'p', '', '', 'p', 'q', ''],
 ['', '', '', '', '', '', '', 'p'],
 ['', '', '', '', '', '', '', 'P'],
 ['', '', '', 'r', 'Q', 'N', '', ''],
 ['', '', '', '', '', '', '', 'K'],
 ['', '', '', '', '', 'P', '', ''],
 ['', '', '', '', '', '', '', '']]
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [

In [ ]:

def torch_log(x: torch.Tensor) -> torch.Tensor:
    """ torch.log(0)によるnanを防ぐ． """
    return torch.log(torch.clamp(x, min=1e-10))

class VAE(nn.Module):
    """ VAEモデルの実装 """
    def __init__(self, z_dim: int) -> None:
        """
        クラスのコンストラクタ．

        Parameters
        ----------
        z_dim : int
            VAEの潜在空間の次元数．
        """
        super().__init__()

        # Encoder, xを入力にガウス分布のパラメータmu, sigmaを出力
        self.dense_enc1 = nn.Linear(8*8, 200)
        self.dense_enc2 = nn.Linear(200, 200)
        self.dense_encmean = nn.Linear(200, z_dim)
        self.dense_encvar = nn.Linear(200, z_dim)

        # Decoder, zを入力にベルヌーイ分布のパラメータlambdaを出力
        self.dense_dec1 = nn.Linear(z_dim, 200)
        self.dense_dec2 = nn.Linear(200, 200)
        self.dense_dec3 = nn.Linear(200, 8*8)

    def _encoder(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        VAEのエンコーダ部分．

        Parameters
        ----------
        x : torch.Tensor ( b, c * h * w )
            Flattenされた入力画像．

        Returns
        ----------
        mean : torch.Tensor ( b, z_dim )
            エンコーダがモデリングするガウス分布の平均
        std : torch.Tensor ( b, z_dim )
            エンコーダがモデリングするガウス分布の標準偏差
        """
        x = F.relu(self.dense_enc1(x))
        x = F.relu(self.dense_enc2(x))
        mean = self.dense_encmean(x)
        std = F.softplus(self.dense_encvar(x))

        return mean, std

    def _sample_z(self, mean: torch.Tensor, std: torch.Tensor) -> torch.Tensor:
        """
        訓練時に再パラメータ化トリックによってガウス分布から潜在変数をサンプリングする．
        推論時はガウス分布の平均を返す．

        Parameters
        ----------
        mean : torch.Tensor ( b, z_dim )
            エンコーダがモデリングするガウス分布の平均
        std : torch.Tensor ( b, z_dim )
            エンコーダがモデリングするガウス分布の標準偏差
        """
        if self.training:
            epsilon = torch.randn(mean.shape).to(device)
            return mean + std * epsilon
        else:
            return mean

    def _decoder(self, z: torch.Tensor) -> torch.Tensor:
        """
        VAEのデコーダ部分．

        Parameters
        ----------
        z : torch.Tensor ( b, z_dim )
            潜在変数．

        Returns
        ----------
        x : torch.Tensor ( b, c * h * w )
            再構成画像．
        """
        x = F.relu(self.dense_dec1(z))
        x = F.relu(self.dense_dec2(x))
        # 出力が0~1になるようにsigmoid
        x = torch.sigmoid(self.dense_dec3(x))
        return x

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        順伝播．

        Parameters
        ----------
        x : torch.Tensor ( b, c * h * w )
            Flattenされた入力画像．

        Returns
        ----------
        x : torch.Tensor ( b, c * h * w )
            再構成画像．
        z : torch.Tensor ( b, z_dim )
            潜在変数．
        """
        mean, std = self._encoder(x)
        z = self._sample_z(mean, std)
        x = self._decoder(z)
        return x, z

    def loss(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        順伝播しつつ目的関数の計算を行う．

        Parameters
        ----------
        x : torch.Tensor ( b, c * h * w )
            Flattenされた入力画像．

        Returns
        ----------
        KL : torch.Tensor (, )
            正則化項．エンコーダ（ガウス分布）と事前分布（標準ガウス分布）のKLダイバージェンス．
        reconstruction : torch.Tensor (, )
            再構成誤差．
        """
        mean, std = self._encoder(x)

        # KL loss(正則化項)の計算. mean, stdは (batch_size , z_dim)
        # torch.sumは上式のJ(=z_dim)に関するもの. torch.meanはbatch_sizeに関するものなので,
        # 上式には書いてありません.
        KL = -0.5 * torch.mean(torch.sum(1 + torch_log(std**2) - mean**2 - std**2,dim=1))

        z = self._sample_z(mean, std)
        y = self._decoder(z)

        # reconstruction loss(負の再構成誤差)の計算. x, yともに (batch_size , 784)
        # torch.sumは上式のD(=784)に関するもの. torch.meanはbatch_sizeに関するもの.
        reconstruction = torch.mean(torch.sum(x * torch_log(y) + (1-x) * torch_log(1-y),dim=1))

        return KL, -reconstruction

In [ ]:
def binary_accuracy(y, t):
    pred = y >= 0
    truth = t >= 0.5
    return pred.eq(truth).sum().item() / len(t)

In [ ]:
z_dim = 2
assert z_dim >= 2

model = VAE(z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    losses = []
    KL_losses = []
    reconstruction_losses = []

    model.train()
    for x, _ in train_loader:

        x = x.to(device)

        model.zero_grad()

        # KL_loss, reconstruction_lossの各項の計算
        KL_loss, reconstruction_loss = model.loss(x)

        # エビデンス下界の最大化のためマイナス付きの各項の値を最小化するようにパラメータを更新
        loss = KL_loss + reconstruction_loss

        loss.backward()
        optimizer.step()

        losses.append(loss.cpu().detach().numpy())
        KL_losses.append(KL_loss.cpu().detach().numpy())
        reconstruction_losses.append(reconstruction_loss.cpu().detach().numpy())

    losses_val = []
    model.eval()
    for x, t in valid_loader:

        x = x.to(device)

        KL_loss, reconstruction_loss = model.loss(x)

        loss = KL_loss + reconstruction_loss

        losses_val.append(loss.cpu().detach().numpy())

    print('EPOCH: %d    Train Lower Bound: %lf (KL_loss: %lf. reconstruction_loss: %lf)    Valid Lower Bound: %lf' %
          (epoch+1, np.average(losses), np.average(KL_losses), np.average(reconstruction_losses), np.average(losses_val)))

In [ ]:
#ネットワーク化の逆の処理

import torch

# board_list から PyTorch テンソルに変換する関数
def board_list_to_tensor(board_list):
    tensor_list = []
    for row in board_list:
        tensor_row = []
        for char in row:
            if char == "":
                tensor_row.extend([0] * len(piece_list))
            else:
                for i, piece in enumerate(piece_list):
                    if char.upper() == piece.upper():
                        tensor_row.extend([1 if i == j else 0 for j in range(len(piece_list))])
        tensor_list.append(tensor_row)
    return torch.tensor(tensor_list, dtype=torch.int8)

# テンソルから board_list に変換する関数
def tensor_to_board_list(tensor):
    board_list = []
    for row in tensor:
        row_list = []
        for i, value in enumerate(row):
            if value == 1:
                row_list.append(piece_list[i].lower())
            else:
                row_list.append("")
        board_list.append(row_list)
    return board_list

# テンソルに変換
tensor_board = board_list_to_tensor(board_list)
print("Tensor representation:")
print(tensor_board)

# board_list に戻す
back_to_board_list = tensor_to_board_list(tensor_board)
print("\nBack to board_list:")
pprint(back_to_board_list)
